# Arrow

## Neural Net Predicion

## Baseline

In [ ]:
# import required libraries
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
import random

In [ ]:
# import Data
Data = pd.read_csv('https://drive.google.com/u/0/uc?id=1V_xIhHtXbB_QbjqNvfiHinLjejhZbqaw&export=download', index_col=0)
#Data = pd.read_csv('LEADTIME.csv', index_col=0)
print(Data.shape)
#Data.head(20).T
Data.head(5)

## Exploratory Data Analysis (EDA)

In [ ]:
#get some information about our Data-Set
Data.info()
# check if there are any Null values
Data.isnull().sum()

In [ ]:
print(Data.columns)
columnList = Data.columns.tolist()
print('mfr  = ', len(Data.mfr.unique()))
print('icc3 = ', len(Data.icc3.unique()))
print('icc2 = ', len(Data.icc2.unique()))
print('icc1 = ', len(Data.icc1.unique()))
print('mpn  = ', len(Data.mpn.unique()))

In [ ]:
Data.describe().transpose()
#Data.describe()

In [ ]:
# visualizing actualleadtime
fig = plt.figure(figsize=(10,7))
fig.add_subplot(2,1,1)
sns.distplot(x=Data['actualleadtime'])
fig.add_subplot(2,1,2)
sns.boxplot(x=Data['actualleadtime'])
plt.tight_layout()

In [ ]:
# visualizing expected_leadtime
fig = plt.figure(figsize=(10,7))
fig.add_subplot(2,1,1)
sns.distplot(x=Data['expected_leadtime'])
fig.add_subplot(2,1,2)
sns.boxplot(x=Data['expected_leadtime'])
plt.tight_layout()

In [ ]:
# Arrow's Expected Leadtime (MAE - MSE - RMSE)
print('MAE:      ', metrics.mean_absolute_error(Data['actualleadtime'], Data['expected_leadtime']))  
print('MSE:      ', metrics.mean_squared_error(Data['actualleadtime'], Data['expected_leadtime']))  
print('RMSE:     ', np.sqrt(metrics.mean_squared_error(Data['actualleadtime'], Data['expected_leadtime'])))
print('VarScore: ', metrics.explained_variance_score(Data['actualleadtime'],Data['expected_leadtime']))
print('R^2:      ', metrics.r2_score(Data['actualleadtime'], Data['expected_leadtime']))

In [ ]:
# visualizing expected_leadtime - continuous

fig = plt.figure(figsize=(10,7))
sns.scatterplot(x=Data['expected_leadtime'],y=Data['actualleadtime'], alpha=0.1)
plt.axis('square')
plt.tight_layout()

In [ ]:
EconData = Data.copy(deep=True)
EconData['error_expected_leadtime'] = Data['expected_leadtime'] - Data['actualleadtime']
EconData['expected_leadtime_cost'] = np.nan

In [ ]:
#Error of 'expected'
#Negative is too early
EconData = Data.copy(deep=True)
EconData['error_expected_leadtime'] = Data['expected_leadtime'] - Data['actualleadtime']

#Create cost for 'expected_leadtime'
EconData['expected_leadtime_cost'] = np.nan

for index, row in EconData.iterrows():
    if (EconData['error_expected_leadtime'][index] > -1) & (EconData['error_expected_leadtime'][index] < 1): 
    # -1 to 1 weeks, $0
        EconData['expected_leadtime_cost'][index] = 0.00
    elif (EconData['error_expected_leadtime'][index] >= 1) & (EconData['error_expected_leadtime'][index] < 2): 
    # 1 week, $400
        EconData['expected_leadtime_cost'][index] = 400.00
    elif (EconData['error_expected_leadtime'][index] >= 2) & (EconData['error_expected_leadtime'][index] < 4): 
    # 2 weeks - 3 weeks, $600
        EconData['expected_leadtime_cost'][index] = 600.00  
    elif (EconData['error_expected_leadtime'][index] >= 4) & (EconData['error_expected_leadtime'][index] < 8): 
    # 4 weeks - 7 weeks, $1,000 
        EconData['expected_leadtime_cost'][index] = 1000.00
    elif (EconData['error_expected_leadtime'][index] >= 8): 
    # 8 weeks or more, $4,000
        EconData['expected_leadtime_cost'][index] = 4000.00      
    else: 
    # all negative weeks (too early), $400/week 
        EconData['expected_leadtime_cost'][index] = np.floor(abs(EconData['error_expected_leadtime'][index]))*400.00

        
import locale
locale.setlocale( locale.LC_ALL, '' )
print('Associated costs from using Expected as warehouse guidance:  ' + str(locale.currency(EconData['expected_leadtime_cost'].sum(), grouping=True )))


In [ ]:
# visualizing PCA - continuous

fig = plt.figure(figsize=(16,80))
for i,pca in enumerate(columnList[7:]):
    name = '%s' %(pca)
    counter = 1+i
    fig.add_subplot(21,2,counter)
    sns.scatterplot(x=Data[name],y=Data['actualleadtime'], alpha=0.25)
plt.tight_layout()

## Transform hashed names to human readable

In [ ]:
# categorical - convert hashed names to ints

mfr_hashes = Data.mfr.unique()
mfr_dict = dict(zip(mfr_hashes, range(len(mfr_hashes))))
# map ints to hashed names with dictionary
Data['mfr'] = Data['mfr'].map(mfr_dict)

icc3_hashes = Data.icc3.unique()
icc3_dict = dict(zip(icc3_hashes, range(len(icc3_hashes))))
# map ints to hashed names with dictionary
Data['icc3'] = Data['icc3'].map(icc3_dict)

icc2_hashes = Data.icc2.unique()
icc2_dict = dict(zip(icc2_hashes, range(len(icc2_hashes))))
# map ints to hashed names with dictionary
Data['icc2'] = Data['icc2'].map(icc2_dict)

icc1_hashes = Data.icc1.unique()
icc1_dict = dict(zip(icc1_hashes, range(len(icc1_hashes))))
# map ints to hashed names with dictionary
Data['icc1'] = Data['icc1'].map(icc1_dict)

mpn_hashes = Data.mpn.unique()
mpn_dict = dict(zip(mpn_hashes, range(len(mpn_hashes))))
# map ints to hashed names with dictionary
Data['mpn'] = Data['mpn'].map(mpn_dict)

In [ ]:
# visualizing categorical

fig = plt.figure(figsize=(15,24))
for i,label in enumerate(columnList[:4]):
    name = '%s' %(label)
    counter = 1+i
    fig.add_subplot(4,1,counter)
    sns.countplot(x=Data[name])
    plt.xticks(rotation=90)
plt.tight_layout()

In [ ]:
# correlations with target

print(Data[Data.columns[:]].corr()['actualleadtime'][:].sort_values())

In [ ]:
# Compute the correlation matrix
corr_mat = Data.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_mat, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_mat, mask=mask, cmap=cmap, vmax=.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
# Retain upper triangular values of correlation matrix and 
# make Lower triangular values Null 
upper_corr_mat = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))

# Convert to 1-D series and drop Null values 
unique_corr_pairs = upper_corr_mat.unstack().dropna() 

# Sort correlation pairs 
sorted_mat = unique_corr_pairs.sort_values() 
print(sorted_mat)

## Dataset Preparation for modeling Train - Test

In [ ]:
X = Data.drop('actualleadtime',axis =1).values
y = Data['actualleadtime'].values

In [ ]:
#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

## Baseline Keras Neural Net

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

model1 = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[inputShape]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model1.compile(
    optimizer='adam',
    loss='mae',
)

history1 = model1.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=10,
)

import pandas as pd

# convert the training history to a dataframe
history_df = pd.DataFrame(history1.history)
# use Pandas native plot method
history_df['loss'].plot();

y_pred = model1.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')

#MAE:      6.567314748970678
#MSE:      69.34229602211707
#RMSE:     8.32720217252572
#VarScore: 0.012679280153120365
#R^2     : -0.04580826813625527

#MAE:      6.5665305169940025
#MSE:      75.0684391461496
#RMSE:     8.664204472780499
#VarScore: 0.033679731998591644
#R^2     : -0.13216894794032097

#MAE:      6.687301871523192
#MSE:      67.04483540831164
#RMSE:     8.188091072301996
#VarScore: -0.0005407931594825222
#R^2     : -0.011158372712129827

#MAE:      6.252205755541979
#MSE:      65.47076853783466
#RMSE:     8.091400900822716
#VarScore: 0.04956918355722428
#R^2     : 0.0125814259703243

#MAE:      6.588257200177496
#MSE:      70.9934218506885
#RMSE:     8.425759422787273
#VarScore: 0.009269269799383695
#R^2     : -0.07071025642205653
MAE:      6.589789192545757
MSE:      70.68138430265022
RMSE:     8.407222151379742
VarScore: -0.01357110655237026
R^2     : -0.06600416120416375
[<matplotlib.lines.Line2D at 0x7f0dc42f7a50>]

## Best Keras Neural Net - with extra capacity and early stopping

In [ ]:
X = Data.drop('actualleadtime',axis =1).values
y = Data['actualleadtime'].values

In [ ]:
#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

In [ ]:
inputShape = X_train.shape[1]
print(X_train.shape)

In [ ]:
from tensorflow.keras import callbacks

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)

model2 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

In [ ]:
model2.compile(
    optimizer='adam',
    loss='mae',
)

In [ ]:
history2 = model2.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

In [ ]:
# convert the training history to a dataframe
history_df = pd.DataFrame(history2.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
y_pred = model2.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')

#MAE:      5.652554930816247
#MSE:      55.68107060033343
#RMSE:     7.461974979878546
#VarScore: 0.17452890869855564
#R^2     : 0.1602279221626297

#MAE:      5.6478783102414605
#MSE:      56.02362755063129
#RMSE:     7.4848932891946625
#VarScore: 0.17870092918037417
#R^2     : 0.1550615387072164

#MAE:      5.662848275504706
#MSE:      56.65744416611105
#RMSE:     7.527113933381841
#VarScore: 0.17237954737159078
#R^2     : 0.14550242839538807

#MAE:      5.677067341143838
#MSE:      56.23021178304301
#RMSE:     7.498680669494002
#VarScore: 0.1725942642626136
#R^2     : 0.1519458717807297

#MAE:      5.665296212453778
#MSE:      56.84314413868254
#RMSE:     7.539439245639064
#VarScore: 0.1739490095662677
#R^2     : 0.1427017341892718
MAE:      5.661731753986324
MSE:      56.40033203140934
RMSE:     7.5100154481471835
VarScore: 0.17293202503610794
R^2     : 0.14938014822490508
[<matplotlib.lines.Line2D at 0x7f0dbc157650>]

## Feature Engineering

In [ ]:
# Categorical feature - APPROACH : create 'merged number (e.g, social security number)' 

Data['globalNum'] = np.nan

for i in range(Data.shape[0]):
    Data['globalNum'][i] = int(format(Data['mfr'][i], '03d') + format(Data['icc3'][i], '03d') + 
                               format(Data['icc2'][i], '02d') + format(Data['icc1'][i], '02d'))

print('globalNum  = ', len(Data.globalNum.unique()))
#Data[['mfr', 'icc3', 'icc2','icc1','globalNum']].tail(6)

In [ ]:
# Categorical feature - APPROACH : create 'counts'                             

valueCounts = Data['mfr'].value_counts()    
valueCounts.name = 'mfr_counts'
Data = Data.join(valueCounts, on='mfr')

valueCounts = Data['icc3'].value_counts()    
valueCounts.name = 'icc3_counts'
Data = Data.join(valueCounts, on='icc3')

valueCounts = Data['icc2'].value_counts()    
valueCounts.name = 'icc2_counts'
Data = Data.join(valueCounts, on='icc2')

valueCounts = Data['icc1'].value_counts()    
valueCounts.name = 'icc1_counts'
Data = Data.join(valueCounts, on='icc1')

valueCounts = Data['mpn'].value_counts()    
valueCounts.name = 'mpn_counts'
Data = Data.join(valueCounts, on='mpn')

valueCounts = Data['globalNum'].value_counts()    
valueCounts.name = 'globalNum_counts'
Data = Data.join(valueCounts, on='globalNum')

#Data.head(1)

In [ ]:
# Categorical feature - QC with NN run
X = Data.drop('actualleadtime',axis =1).values
y = Data['actualleadtime'].values

#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)

model3 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model3.compile(
    optimizer='adam',
    loss='mae',
)

history3 = model3.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

# convert the training history to a dataframe
history_df = pd.DataFrame(history3.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

y_pred = model3.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')

#(52428, 55)
#Minimum validation loss: 6.578306594156533
#MAE:      6.5783064010132914
#MSE:      68.08410929148032
#RMSE:     8.251309550094478
#VarScore: 0.0063678189506687355
#R^2     : -0.02683251796294761

#Minimum validation loss: 6.580244954558053
#MAE:      6.5802450677373
#MSE:      68.44089438108227
#RMSE:     8.272901207018144
#VarScore: 0.00516675247150955
#R^2     : -0.03221348770375987

#(52428, 55)
#Minimum validation loss: 6.584634757054665
#MAE:      6.585235028169771
#MSE:      68.48932255208638
#RMSE:     8.275827605266217
#VarScore: 0.0038829261789706804
#R^2     : -0.032943872830186294

#(52428, 55)
#Minimum validation loss: 6.5619874945710635
#MAE:      6.561987362355801
#MSE:      68.12458229036086
#RMSE:     8.253761705450483
#VarScore: 0.00871068632673544
#R^2     : -0.027442924587672834
(52428, 55)
Minimum validation loss: 6.581878185272217
MAE:      6.5818781666000055
MSE:      68.46174441464751
RMSE:     8.274161251428904
VarScore: 0.0050394398516169225
R^2     : -0.03252794422949279
[<matplotlib.lines.Line2D at 0x7f0dbded3650>]

Hurts model big time, but, what changed? Look at correlations.

In [ ]:
# Print correlations with target
print(Data[Data.columns[:]].corr()['actualleadtime'][:].sort_values())

In [ ]:
# Create subset of best correlated features
Datadeepcopy = Data.copy(deep=True)

good_correlations = Data[Data.columns[:]].corr()['actualleadtime'][:].sort_values()
good_corr_keys = good_correlations[abs(good_correlations) > 0.05].keys()
print(good_corr_keys.shape)
print(good_corr_keys)
Data_corr = Data[good_corr_keys]

In [ ]:
# RERUN NN with categorical standardization step with reduced features from correlations - QC
X = Data_corr.drop('actualleadtime',axis =1).values
y = Data_corr['actualleadtime'].values

#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)

model4 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model4.compile(
    optimizer='adam',
    loss='mae',
)

history4 = model4.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

# convert the training history to a dataframe
history_df = pd.DataFrame(history4.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

y_pred = model4.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')

#(52428, 9)
#Minimum validation loss: 5.394649815539378
#MAE:      5.394649885707965
#MSE:      52.18399158458135
#RMSE:     7.223848806874446
#VarScore: 0.22581255892630892
#R^2     : 0.21297025056538155

#(52428, 9)
#Minimum validation loss: 5.435182109526296
#MAE:      5.435285187908102
#MSE:      54.682551431700176
#RMSE:     7.39476513702093
#VarScore: 0.21817158964288386
#R^2     : 0.17528741200294484

#(52428, 9)
#Minimum validation loss: 5.472837763028368
#MAE:      5.472974350643965
#MSE:      54.83584102127504
#RMSE:     7.405122620272742
#VarScore: 0.20655990802351176
#R^2     : 0.17297552547202488

#(52428, 9)
#Minimum validation loss: 5.4677299398519885
#MAE:      5.467729999679463
#MSE:      54.68307986610659
#RMSE:     7.3948008672381835
#VarScore: 0.20336280659293415
#R^2     : 0.1752794422485079

#(52428, 9)
#Minimum validation loss: 5.451549962118633
#MAE:      5.451550021591399
#MSE:      53.755060736786604
#RMSE:     7.331784280568176
#VarScore: 0.2139574314112922
#R^2     : 0.18927566294073717
(52428, 9)
Minimum validation loss: 5.472656726837158
MAE:      5.4726566950316435
MSE:      54.476041767043434
RMSE:     7.380788695460901
VarScore: 0.20552668555365938
R^2     : 0.1784019543117138
[<matplotlib.lines.Line2D at 0x7f0dc4948090>]

OK, so it's safe to say we can drop features to get improvement

In [ ]:
# Numerical feature engineering
# PCA standardization step 1 - determine which columns are PCA components
#Data.columns
Data.head(1)

In [ ]:
# PCA standardization step 2 - create list of PCA component columns
list = [*range(6, 48, 1)]
#print(list)
#Restore X back to full range of features
X = Data.drop('actualleadtime',axis =1).values
y = Data['actualleadtime'].values

#QC
#print(X[0, 6:48])
#X[0, list]
#columnListPCA = Data.columns.tolist()

#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

In [ ]:
# PCA standardization step 3 - standardize PCA components
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[('num', RobustScaler(), list)], remainder='passthrough')
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

In [ ]:
# PCA standardization step 4 - QC
inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)

model5 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model5.compile(
    optimizer='adam',
    loss='mae',
)

history5 = model5.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

# convert the training history to a dataframe
history_df = pd.DataFrame(history5.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

y_pred = model5.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')


#(52428, 55)
#Minimum validation loss: 6.5822722340117625
#MAE:      6.583061388649056
#MSE:      68.36428226026061
#RMSE:     8.268269604957291
#VarScore: 0.004141823261248345
#R^2     : -0.031058037221283863

#Minimum validation loss: 6.579778779969114
#MAE:      6.579778759241413
#MSE:      68.33070947553436
#RMSE:     8.266239137330492
#VarScore: 0.004770350819703895
#R^2     : -0.03055169840839156

#(52428, 55)
#Minimum validation loss: 6.578337165395665
#MAE:      6.578565093263969
#MSE:      68.48308737984802
#RMSE:     8.27545088680055
#VarScore: 0.004868143088059829
#R^2     : -0.03284983506313499

#(52428, 55)
#Minimum validation loss: 6.583355809545921
#MAE:      6.583355940078592
#MSE:      68.39729229848054
#RMSE:     8.270265551871992
#VarScore: 0.0049065016496776526
#R^2     : -0.03155588879070548

#(52428, 55)
#Minimum validation loss: 6.5828712485134355
#MAE:      6.582871302401818
#MSE:      68.44929316405572
#RMSE:     8.27340879952488
#VarScore: 0.004037667035531434
#R^2     : -0.03234015666599199
(52428, 55)
Minimum validation loss: 6.585849285125732
MAE:      6.585848741548834
MSE:      68.5219444528118
RMSE:     8.277798285342051
VarScore: 0.0038387341517374107
R^2     : -0.03343587057843056
[<matplotlib.lines.Line2D at 0x7f0dbc358990>]

But, we know that we had a better group of features to model,so lets use these again with the RobustScalar applied to the PCA components.

In [ ]:
# Categorical standardization step with reduced features from correlations - QC
X = Data_corr.drop('actualleadtime',axis =1).values
y = Data_corr['actualleadtime'].values

print(Data_corr.columns)
corrcolumnList = Data_corr.columns.tolist()
corrdeepcopy = Data_corr.copy(deep=True)

#
list2 = [0, 1, 2, 6]
#print(X[0, list2])
#print(Data_corr.head(1))

#splitting Train and Test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=50)

# PCA standardization step 3 - standardize PCA components
from sklearn.preprocessing import RobustScaler
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer(transformers=[('num', RobustScaler(), list2)], remainder='passthrough')
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

In [ ]:
# Corr - PCA standardization - QC
inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)

model6 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model6.compile(
    optimizer='adam',
    loss='mae',
)

history6 = model6.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

# convert the training history to a dataframe
history_df = pd.DataFrame(history6.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

y_pred = model6.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')


#(52428, 9)
#Minimum validation loss: 5.438045465255881
#MAE:      5.438045402472503
#MSE:      54.70697851738233
#RMSE:     7.396416599771968
#VarScore: 0.20832859916762636
#R^2     : 0.17491900700861462

#(52428, 9)
#Minimum validation loss: 5.450207921272104
#MAE:      5.4511751146149905
#MSE:      53.2457551332528
#RMSE:     7.296968900389586
#VarScore: 0.21203891300610045
#R^2     : 0.19695692014938082

#(52428, 9)
#Minimum validation loss: 5.442438985787186
#MAE:      5.4424390595847125
#MSE:      54.24873196131605
#RMSE:     7.365373850750283
#VarScore: 0.21469537219145252
#R^2     : 0.1818302006764193
(52428, 9)
Minimum validation loss: 5.454400062561035
MAE:      5.454593881249583
MSE:      54.32281156262853
RMSE:     7.370401044897661
VarScore: 0.2126950660502207
R^2     : 0.18071294520613423
[<matplotlib.lines.Line2D at 0x7f0dbe2966d0>]

In [ ]:
# Now run with validation approach
inputShape = X_train.shape[1]
print(X_train.shape)

keras.backend.clear_session()
random.seed(10)

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,                   # minimium amount of change to count as an improvement
    patience=20,                       # how many epochs to wait before stopping
    restore_best_weights=True,
)
 
model7 = keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[inputShape]), 
    layers.Dense(1024, activation='relu'),
    layers.Flatten(),
    layers.Dense(1),
])

model7.compile(
    optimizer='adam',
    loss='mae',
)

history7 = model7.fit(
    X_train, y_train,
#    validation_data=(X_test, y_test),
    validation_split=0.20, 
    shuffle=True, 
    batch_size=256,
    epochs=500,
    callbacks=[early_stopping],        # put your callbacks in a list
    verbose=0,                         # turn off training log
)

# convert the training history to a dataframe
history_df = pd.DataFrame(history7.history)
# use Pandas native plot method
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

y_pred = model7.predict(X_test)
print('MAE:     ', metrics.mean_absolute_error(y_test, y_pred))  
print('MSE:     ', metrics.mean_squared_error(y_test, y_pred))  
print('RMSE:    ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('VarScore:',metrics.explained_variance_score(y_test,y_pred))
print('R^2     :', metrics.r2_score(y_test, y_pred))
# Visualizing Our predictions
fig = plt.figure(figsize=(10,5))
plt.scatter(y_test,y_pred)
# Perfect predictions
plt.plot(y_test,y_test,'r')


#(52428, 9)
#Minimum validation loss: 5.406984850079327
#MAE:      5.464944444405202
#MSE:      53.89007596127871
#RMSE:     7.340986034674001 - @256
#VarScore: 0.2078638606317419
#R^2     : 0.18723938715816035

#Minimum validation loss: 5.655806328898824
#MAE:      5.73160720036142
#MSE:      59.1501619756141
#RMSE:     7.69091424835917
#VarScore: 0.1531305732989643
#R^2     : 0.10790769841302206


#(52428, 9)
#Minimum validation loss: 5.40487624387943
#MAE:      5.4650467362530835
#MSE:      53.89647817690637
#RMSE:     7.341422081375404
#VarScore: 0.20204994398927567
#R^2     : 0.18714283007219856

#(52428, 9)
#Minimum validation loss: 5.490564767308938
#MAE:      5.569218715786834
#MSE:      55.52829144063706
#RMSE:     7.451730768126091
#VarScore: 0.18194490853407075
#R^2     : 0.16253211048022476

#(52428, 9)
#Minimum validation loss: 5.450915723317183
#MAE:      5.527843021424255
#MSE:      55.040430510997965
#RMSE:     7.4189238108365805
#VarScore: 0.19404366937924367
#R^2     : 0.1698899429026537


(52428, 9)
Minimum validation loss: 5.402743816375732
MAE:      5.4759828102818755
MSE:      54.1524600662066
RMSE:     7.358835510201774
VarScore: 0.20499657311212205
R^2     : 0.18328215640431278
[<matplotlib.lines.Line2D at 0x7f0dbdccfc50>]

### Economic Model

In [ ]:
econ = pd.DataFrame(X_test)
econ['actual'] = y_test
econ['pred'] = y_pred 

In [ ]:
Data_corr.head()

In [ ]:
#econ[8]
econ
econ = econ.rename(columns={8: 'expected'})
econ

In [ ]:
#Error of model
#Negative is too early
econ['error_model'] = econ['pred'] - econ['actual']
#Error of 'expected'
#Negative is too early
econ['error_expected'] = econ['expected'] - econ['actual']

In [ ]:
#Create cost for 'expected_leadtime'
econ['expected_leadtime_cost'] = np.nan

for index, row in econ.iterrows():
    if (econ['error_expected'][index] > -1) & (econ['error_expected'][index] < 1): 
    # -1 to 1 weeks, $0
        econ['expected_leadtime_cost'][index] = 0.00
    elif (econ['error_expected'][index] >= 1) & (econ['error_expected'][index] < 2): 
    # 1 week, $400
        econ['expected_leadtime_cost'][index] = 400.00
    elif (econ['error_expected'][index] >= 2) & (econ['error_expected'][index] < 4): 
    # 2 weeks - 3 weeks, $600
        econ['expected_leadtime_cost'][index] = 600.00  
    elif (econ['error_expected'][index] >= 4) & (econ['error_expected'][index] < 8): 
    # 4 weeks - 7 weeks, $1,000 
        econ['expected_leadtime_cost'][index] = 1000.00
    elif (econ['error_expected'][index] >= 8): 
    # 8 weeks or more, $4,000
        econ['expected_leadtime_cost'][index] = 4000.00      
    else: 
    # all negative weeks (too early), $400/week 
        econ['expected_leadtime_cost'][index] = np.floor(abs(econ['error_expected'][index]))*400.00


In [ ]:
#Create cost for model
econ['modeledcost'] = np.nan

for index, row in econ.iterrows():
    if (econ['error_model'][index] > -1) & (econ['error_model'][index] < 1): 
    # -1 to 1 weeks, $0
        econ['modeledcost'][index] = 0.00
    elif (econ['error_model'][index] >= 1) & (econ['error_model'][index] < 2): 
    # 1 week, $400
        econ['modeledcost'][index] = 400.00
    elif (econ['error_model'][index] >= 2) & (econ['error_model'][index] < 4): 
    # 2 weeks - 3 weeks, $600
        econ['modeledcost'][index] = 600.00  
    elif (econ['error_model'][index] >= 4) & (econ['error_model'][index] < 8): 
    # 4 weeks - 7 weeks, $1,000 
        econ['modeledcost'][index] = 1000.00
    elif (econ['error_model'][index] >= 8): 
    # 8 weeks or more, $4,000
        econ['modeledcost'][index] = 4000.00      
    else: 
    # all negative weeks (too early), $400/week 
        econ['modeledcost'][index] = np.floor(abs(econ['error_model'][index]))*400.00

In [ ]:
#Data_for_econ.head(10)
#np.floor(Data_for_econ['error_modeledleadtime'].iloc[1])
(econ['expected_leadtime_cost'].sum() - econ['modeledcost'].sum()) / econ['expected_leadtime_cost'].sum()

In [ ]:
import locale
locale.setlocale( locale.LC_ALL, '' )
print('RMSE:                     ', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAE:                      ', metrics.mean_absolute_error(y_test, y_pred))  
print('Expected:                  ' + str(locale.currency(econ['expected_leadtime_cost'].sum(), grouping=True )))
print('Neural Net:                ' + str(locale.currency(econ['modeledcost'].sum(), grouping=True )))
print('Savings from using model:  ' + str(locale.currency((econ['expected_leadtime_cost'].sum() - econ['modeledcost'].sum()), grouping=True )))
print('%Savings from using model: ' + str((econ['expected_leadtime_cost'].sum() - econ['modeledcost'].sum()) / econ['expected_leadtime_cost'].sum()))

#Neural Net:               $115,159,600.00
#Expected                  $148,529,400.00
#Savings from using model: $33,369,800.00



RMSE:                      7.358835510201774
MAE:                       5.4759828102818755
Expected:                  $29,717,400.00
Neural Net:                $23,376,000.00
Savings from using model:  $6,341,400.00
%Savings from using model: 0.21339013507238183

In [ ]:
print('RMSE OF ARROW: ', np.sqrt(metrics.mean_squared_error(econ['actual'], econ['expected'])))

In [ ]:
RMSE OF ARROW:  10.065229386516192

### END